In [58]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
os.environ['GEMINI_API_KEY']=os.getenv("GEMINI_API_KEY")
# os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")

In [60]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core import Settings
import chromadb

In [61]:
# from llama_index.llms.groq import Groq
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding

In [62]:
llm=Gemini(api_key=os.environ["GEMINI_API_KEY"])
embed_model=GeminiEmbedding(model_name="models/embedding-001",api_key=os.environ["GEMINI_API_KEY"])


In [ ]:
# from llama_index.core.ingestion import IngestionPipeline
# from llama_index.core.node_parser import SentenceSplitter
# from llama_index.core.retrievers import VectorContextRetriever
# from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore

In [ ]:
# from llama_index.core import SimpleDirectoryReader
# documents=SimpleDirectoryReader("data").load_data()

In [65]:
import pdfplumber

def extract_pdf_content(pdf_path):
    text_content = ""
    tables = []

    with pdfplumber.open(pdf_path) as pdf:
        # Extract text from all pages
        for page in pdf.pages:
            text_content += page.extract_text() + "\n"  # Extract text
        
        # Extract tables from each page
        for page in pdf.pages:
            page_tables = page.extract_tables()
            for table in page_tables:
                tables.append(table)
    
    return text_content, tables

# Convert the extracted tables into a string format suitable for LlamaIndex
def format_tables_as_text(tables):
    table_text = ""
    for table in tables:
        for row in table:
            # Handle None values in the table row by replacing with empty string
            row = [str(cell) if cell is not None else "" for cell in row]
            table_text += "\t".join(row) + "\n"  # Tab-separated row
        table_text += "\n---\n"  # Separator between tables (optional)
    return table_text

# Extract content and tables from a PDF
pdf_path = "data/paper.pdf"
text_content, tables = extract_pdf_content(pdf_path)

# Format tables into text
table_text = format_tables_as_text(tables)

# Combine text and table data into a single document
combined_text = text_content + "\n\n" + "Tables extracted from the PDF:\n" + table_text

# Now, we can treat `combined_text` as a document to load into LlamaIndex
documents = [combined_text]


In [66]:
# Load from disk
load_client = chromadb.PersistentClient(path="./chroma_db")

# Fetch the collection
chroma_collection = load_client.get_collection("quickstart")

# Fetch the vector store
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

Settings.llm = llm
Settings.embed_model = embed_model

# Get the index from the vector store
index = VectorStoreIndex.from_vector_store(
    vector_store
)

test_query_engine = index.as_query_engine()
response = test_query_engine.query("give me brief about the text")
print(response)

The text discusses future improvements to an IoT-based health monitoring system.  These include adding more health metrics, proactive wellness management features with statistical notifications, a multi-modal approach for various age groups, personalized fitness recommendations, and long-term progress tracking.  The goal is to create a system that significantly improves remote health management.



In [67]:
import pdfplumber
from llama_index.core import Document,VectorStoreIndex, ServiceContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.storage.storage_context import StorageContext
import chromadb
import os

# Step 1: Extract text and tables from the PDF using pdfplumber
def extract_pdf_content(pdf_path):
    text_content = ""
    tables = []

    with pdfplumber.open(pdf_path) as pdf:
        # Extract text from all pages
        for page in pdf.pages:
            text_content += page.extract_text() + "\n"  # Extract text
        
        # Extract tables from each page
        for page in pdf.pages:
            page_tables = page.extract_tables()
            for table in page_tables:
                tables.append(table)
    
    return text_content, tables

# Step 2: Convert the extracted tables into a string format suitable for LlamaIndex
def format_tables_as_text(tables):
    table_text = ""
    for table in tables:
        for row in table:
            # Handle None values in the table row by replacing with empty string
            row = [str(cell) if cell is not None else "" for cell in row]
            table_text += "\t".join(row) + "\n"  # Tab-separated row
        table_text += "\n---\n"  # Separator between tables (optional)
    return table_text

# Step 3: Create a ChromaDB client and collection
client = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = client.get_or_create_collection("pdf_documents")

# Step 4: Create the vector store using ChromaDB
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Step 5: Extract text and tables from the PDF
pdf_path = "data/paper.pdf"
text_content, tables = extract_pdf_content(pdf_path)

# Step 6: Format tables into text and combine them with the main text content
table_text = format_tables_as_text(tables)
combined_text = text_content + "\n\n" + "Tables extracted from the PDF:\n" + table_text

# Step 7: Wrap the combined text into LlamaIndex BaseDocument format
documents = [Document(text=combined_text, doc_id="pdf_document_1")]

# Step 8: Create the storage context with the ChromaVectorStore
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Step 9: Create an index from the documents and store it in ChromaDB
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

# Optionally save the index to disk
# index.save_to_disk("index_path")

# Step 10: Query the index for specific data
query = "Find tables and content in the given data"
response = index.as_query_engine().query(query)
print("Query Result:", response)

# # Optionally: You can perform a query on ChromaDB directly if you need more flexibility:
# results = chroma_collection.query(query_texts=[query], n_results=1)
# print("ChromaDB Query Result:", results)


Query Result: The data shows a comparison of three machine learning models: Linear Regression, Decision Tree Regressor, and XGBoost Regressor.  Each model's performance is evaluated using the R-squared score (R2 Score) and Mean Squared Error (MSE).  The XGBoost Regressor achieved the highest R2 score (0.9674) and the lowest MSE (0.0033), indicating the best performance among the three.



In [68]:
# Step 1: Extract text and tables from the PDF using pdfplumber
def extract_pdf_content(pdf_path):
    text_content = ""
    tables = []

    with pdfplumber.open(pdf_path) as pdf:
        # Extract text from all pages
        for page in pdf.pages:
            text_content += page.extract_text() + "\n"  # Extract text
        
        # Extract tables from each page
        for page in pdf.pages:
            page_tables = page.extract_tables()
            for table in page_tables:
                tables.append(table)
    
    return text_content, tables

# Step 2: Convert the extracted tables into a string format suitable for LlamaIndex
def format_tables_as_text(tables):
    table_text = ""
    for table in tables:
        for row in table:
            # Handle None values in the table row by replacing with empty string
            row = [str(cell) if cell is not None else "" for cell in row]
            table_text += "\t".join(row) + "\n"  # Tab-separated row
        table_text += "\n---\n"  # Separator between tables (optional)
    return table_text

# Step 3: Create a ChromaDB client and collection
client = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = client.get_or_create_collection("pdf_documents")

# Step 4: Create the vector store using ChromaDB
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Step 5: Extract text and tables from the PDF
pdf_path = "data/paper.pdf"
text_content, tables = extract_pdf_content(pdf_path)

# Step 6: Format tables into text and combine them with the main text content
table_text = format_tables_as_text(tables)

# Step 7: Split documents into separate text and table entries for better indexing
documents = []

# Add text content as a separate document
documents.append(Document(text=text_content, doc_id="pdf_text"))

# Add table content as separate documents (one for each table)
for i, table in enumerate(tables):
    table_doc_text = format_tables_as_text([table])
    documents.append(Document(text=table_doc_text, doc_id=f"pdf_table_{i+1}"))

# Step 8: Create the storage context with the ChromaVectorStore
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Step 9: Create an index from the documents and store it in ChromaDB
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

# Optionally save the index to disk (if you want to persist the index)
# storage_context.persist(persist_dir="./storage")

# Step 10: Query the index for specific data
query = "Find tables and give it exactly as it is"
response = index.as_query_engine().query(query)
print("Query Result:", response)

Query Result: S. No	Machine Learning Model	R2 Score	MSE
1	Linear regression	0.9481	0.0053
2	Decision tree Regressor	0.9574	0.0426
3	XGBoost Regressor	0.9674	0.0033



In [69]:
query = "how many tables"
response = index.as_query_engine().query(query)
print("Query Result:", response)

Query Result: One.

